In [8]:
import mysql.connector
import pandas as pd
import chardet

In [12]:
df = pd.read_csv('../../data/Seoul_CCTV.csv', encoding='UTF-8-SIG')
df.head()

,기관명,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


In [11]:
def detect_csv_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']


csv_file_path = '../../data/Seoul_CCTV.csv'  # 파일 경로를 실제 파일의 경로로 바꿔주세요
encoding = detect_csv_encoding(csv_file_path)

print(f"The encoding of the CSV file is: {encoding}")

The encoding of the CSV file is: UTF-8-SIG


In [28]:
remote = mysql.connector.connect(
    host = 'database-1.cb2iuock61zt.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user =  'root',
    password = 'asdf1234',
    database = 'amrdb'
)

In [29]:
cur = remote.cursor()
cur.execute("CREATE TABLE cctv ( GU varchar(16), SUM int,BEFORE_2013 int ,`2014` int, `2015` int, `2016` int)")#숫자로 시작하는 열 이름을 backtick (`)로 감싸야 합니다.

In [38]:
sql = """insert into cctv values (%s,%s,%s,%s,%s,%s)""" #따옴표를 세 개씩 (""") 사용하는 것은 여러 줄에 걸친 문자열을 정의하는 Python의 표준 문법
cursor = remote.cursor(buffered=True)


In [39]:
for i , row in df.iterrows(): #iterrows()는 DataFrame의 각 행을 (인덱스, 행 데이터)의 튜플로 반환
    cursor.execute(sql, tuple(row)) # execute 메서드를 호출하여 SQL 쿼리를 실행하고, 튜플의 값이 쿼리에 매개변수로 전달되어 데이터베이스에 레코드를 추가
    print(tuple(row))
    remote.commit()

('강남구', 3238, 1292, 430, 584, 932)
('강동구', 1010, 379, 99, 155, 377)
('강북구', 831, 369, 120, 138, 204)
('강서구', 911, 388, 258, 184, 81)
('관악구', 2109, 846, 260, 390, 613)
('광진구', 878, 573, 78, 53, 174)
('구로구', 1884, 1142, 173, 246, 323)
('금천구', 1348, 674, 51, 269, 354)
('노원구', 1566, 542, 57, 451, 516)
('도봉구', 825, 238, 159, 42, 386)
('동대문구', 1870, 1070, 23, 198, 579)
('동작구', 1302, 544, 341, 103, 314)
('마포구', 980, 314, 118, 169, 379)
('서대문구', 1254, 844, 50, 68, 292)
('서초구', 2297, 1406, 157, 336, 398)
('성동구', 1327, 730, 91, 241, 265)
('성북구', 1651, 1009, 78, 360, 204)
('송파구', 1081, 529, 21, 68, 463)
('양천구', 2482, 1843, 142, 30, 467)
('영등포구', 1277, 495, 214, 195, 373)
('용산구', 2096, 1368, 218, 112, 398)
('은평구', 2108, 1138, 224, 278, 468)
('종로구', 1619, 464, 314, 211, 630)
('중구', 1023, 413, 190, 72, 348)
('중랑구', 916, 509, 121, 177, 109)


In [42]:
cursor.execute("select * from cctv")

result = cursor.fetchall()
for row in result:
    print(row)

('강남구', 3238, 1292, 430, 584, 932)
('강동구', 1010, 379, 99, 155, 377)
('강북구', 831, 369, 120, 138, 204)
('강서구', 911, 388, 258, 184, 81)
('관악구', 2109, 846, 260, 390, 613)
('광진구', 878, 573, 78, 53, 174)
('구로구', 1884, 1142, 173, 246, 323)
('금천구', 1348, 674, 51, 269, 354)
('노원구', 1566, 542, 57, 451, 516)
('도봉구', 825, 238, 159, 42, 386)
('동대문구', 1870, 1070, 23, 198, 579)
('동작구', 1302, 544, 341, 103, 314)
('마포구', 980, 314, 118, 169, 379)
('서대문구', 1254, 844, 50, 68, 292)
('서초구', 2297, 1406, 157, 336, 398)
('성동구', 1327, 730, 91, 241, 265)
('성북구', 1651, 1009, 78, 360, 204)
('송파구', 1081, 529, 21, 68, 463)
('양천구', 2482, 1843, 142, 30, 467)
('영등포구', 1277, 495, 214, 195, 373)
('용산구', 2096, 1368, 218, 112, 398)
('은평구', 2108, 1138, 224, 278, 468)
('종로구', 1619, 464, 314, 211, 630)
('중구', 1023, 413, 190, 72, 348)
('중랑구', 916, 509, 121, 177, 109)


In [43]:
df = pd.DataFrame(result)
df.head()

,0,1,2,3,4,5
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


In [47]:
remote.close()